In [1]:
import os, sys
sys.path.append(os.path.join('/home/ring/sumo-svn/', 'tools'))
import traci
import traci.constants as tc
import numpy as np
from sumolib import checkBinary
import datetime

In [2]:
#reset the environment
def reset():
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, "-c", "map_grid_lock_add2TL_copy.sumocfg",
                             "--tripinfo-output", "tripinfo.xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1'])

In [3]:
id = {'virtual' : '270329335' ,'KasemRat': 'cluster_272448137_272555800_272555808_7660045934_7710268409'}
detecters = {'QA' : ['156261350#0'], 'QB' : ['-453962283#3'], 'QC' : ['459551209#0', '459551209#3'], 'QD' : ['27702347#0', '27702347#4', '27702347#6']}

In [4]:
import sumolib
import lxml.etree as ET
def get_edge_id(POI):
    root = ET.Element('additionals')
    net = sumolib.net.readNet('map_grid_lock_add2TL_copy_newTLS.net.xml')
    output = []
    for key in POI:
        edgeID = net.getEdge(key)
        numOfLane = edgeID.getLaneNumber()
        for lane_index in range(numOfLane):
            output.append(key+'_'+str(lane_index))
    return output    

In [5]:
def get_state():
    occupancy = []
    for key in ['QA', 'QB', 'QC', 'QD']:
        occupancy.append(sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(detecters[key])])/len(get_edge_id(detecters[key])))
    current_phase = [traci.trafficlight.getPhase(id[key]) for key in ['virtual','KasemRat']]
    for i in current_phase:
        current_phase_binary = [0, 0, 0]
        current_phase_binary[int(i//2)] = 1
        occupancy+= current_phase_binary
    return occupancy

In [10]:
def initial(max_occupancy):
    occupancy = state[0:4]
    print(occupancy)
    s1 = state[4:7]
    s2 = state[7:9]
    if (occupancy[0] >= max_occupancy) or (occupancy[3] >= max_occupancy):
        print(occupancy)
        print('set0')
        traci.trafficlight.setPhase(id['virtual'], 2)
        if s2[1] == 1:
            traci.trafficlight.setPhase(id['KasemRat'], 0)
    elif occupancy[2] >= max_occupancy:
        print(occupancy)
        print('set2')
        traci.trafficlight.setPhase(id['KasemRat'], 2)
    else:
        traci.trafficlight.setPhase(id['virtual'], 0)
    return

In [13]:
traci.close()

In [12]:
reset()
for seconds in range(10000):
    traci.simulationStep()
    if seconds%5 == 0:
        state = get_state()
        initial(50)
traci.close()

[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 8.81831169220976, 0.0]
[0.0, 0.0, 16.067782894979423, 0.0]
[0.0, 0.0, 16.166341431316717, 0.0]
[0.0, 0.0, 17.899592152070632, 0.0]
[1.8350440624129567, 0.0, 17.52676374436763, 0.0]
[4.279891304347826, 0.0, 21.235858225237195, 0.0]
[7.133152173913045, 6.439473134016308, 22.796659128910978, 0.0]
[8.559782608695654, 11.409742513476138, 19.994779610745667, 0.0]
[9.986413043478262, 14.019908683823, 18.622492776693996, 0.0]
[9.986413043478262, 16.09868283504077, 15.397939185622702, 2.221420843851424]
[9.986413043478262, 16.09868283504077, 18.467009217365618, 4.442841687702848]
[9.986413043478262, 16.09868283504077, 21.93184201676884, 5.629493407041392]
[9.986413043478262, 16.09868283504077, 21.60015148394345, 11.931180900189112]
[9.986413043478262, 16.09868283504077, 18.457990401523453, 17.8968228386569]
[9.986413043478262, 16.09868283504077, 16.065528299267825, 23.215467077912734]
[9.986413043478262, 16.09868283504077, 22.47150963582653, 22.61937378798184]
[9

FatalTraCIError: connection closed by SUMO